In [2]:
#pip install simpledbf 

In [195]:
import pandas as pd
import os
import simpledbf
from simpledbf import Dbf5

In [10]:
%run variables_functions.ipynb

# Data treatment Production
### Coal, Wind, Solar and Natural Gas

In [2]:
df_energy = pd.read_csv("Data/EnergyProduction.csv", header = 0)

def convert_float(string): #problème aperçu quand en train de filtrer pour les renouvelables - il y a d'ans où on a aucune information
    if string == "Not Available":
        return(0)
    else:
        return(float(string))
    
df_energy['Year'] = df_energy['YYYYMM'].apply(lambda x: int(str(x)[:4]))
df_energy['Month'] = df_energy['YYYYMM'].apply(lambda x: int(str(x)[4:6]))
df_energy['Production (MWh)'] = df_energy['Value'].apply(lambda x: convert_float(x)/1000)
df_energy.drop(['Value'], axis = 1, inplace = True)

In [3]:
# Data treatment Coal, Wind, Solar and Natural Gas

energy_index = {'coal':1,'gas':3, 'solar':11, 'wind':12}

def regression(annual_value,year):
    return(annual_value/12)

def df_to_annual(df):
    df = df.where(df['Month']==13).dropna()
    df.drop(['YYYYMM'], axis = 1, inplace = True)
    return(df)

def df_to_month(df):
    for year in range(1949,1973,1):
        for month in range(1,13,1):
            i = (year - 1949)*12 + month - 1
            if month<10:
                yyyymm = int(str(year) + '0' + str(month))
            else:
                yyyymm = int(str(year) + str(month))
            value = df.loc[df['Year']==year]['Production (MWh)'].values[0]
            new_row = pd.DataFrame({'YYYYMM': yyyymm, 'Year': year, 'Month': month, 'Production (MWh)': regression(value,year)}, index = [i])
            df = df.append(new_row, ignore_index = False)
    return(df)

for energy in energy_index:
    df = df_energy.where(df_energy['Column_Order'] == energy_index[energy]).dropna()
    df.drop(['Column_Order','Description','MSN'], axis = 1, inplace = True)
    df_annual = df_to_annual(df)
    df_tool = df_to_month(df)
    df_month = df_tool.where(df_tool['Month']!=13).dropna()
    file_annual = 'treated_data/' + energy + '_treated.csv'
    file_month = 'treated_data/' + energy + '_month_treated.csv'
    df_annual.to_csv(file_annual, sep=';', index=True)
    df_month.to_csv(file_month, sep=';', index=True)


# Data treatment Capacity

### Lecture automatisé de chaque année

##### Création d'un fonction pour traîter le type de fichier 2001-2003
Il a plusieurs en-tetes (headers) et quelques lignes de "NAMEPLATE" ne sont pas en format float

In [4]:
energies = ['Onshore Wind Turbine','Natural Gas Steam Turbine', 'Conventional Steam Coal', 'Solar Photovoltaic']

df_capacity = pd.DataFrame()

for year in year_infos:
    path, energy_source, capacity = year_infos[year][0],year_infos[year][1], year_infos[year][2]

    if year <=2003:
        df_tool = pd.read_csv(path,header = 0)
        df_year_raw = delete_other_headers(df_tool)

    elif year<=2010:
        df_year_raw = pd.read_excel(path,header = 0)
    
    else: 
        df_year_raw = pd.read_excel(path,header = 1)
    
    df_year = df_year_raw.groupby(energy_source, as_index = True).sum()[capacity]
    
    for i in range(len(df_year)):
        code = df_year.index[i]
        if code in codes:
            value = df_year[i]
            new_row = pd.DataFrame({'Year': year,'Energy Source': code, 'Capacity (KW)': value}, index = [i])
            df_capacity = df_capacity.append(new_row, ignore_index = False)

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0,1,5,6,7,22,23,24,25,26) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0,1,5,6,7,17,22,23,24,25,26) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


#### Column "Technology"

In [5]:
df_capacity['Technology'] = df_capacity['Energy Source'].apply(lambda x: codes_technologies[x])

#### Création d'un csv sans l'especification du type d'energie

In [6]:
df_tool = df_capacity.groupby(['Year','Technology']).sum()
df_capacity_general = pd.DataFrame()

for i in range(len(df_tool)): # ici on utilise le range normal, 0 -> 2021-1973, puisque les ans presents dans df_renewable_month sont les indexes
        year = df_tool.iloc[i].name[0]
        technology = df_tool.iloc[i].name[1]
        capacity = df_tool.iloc[i]['Capacity (KW)']
        new_row = pd.DataFrame({'Year': year, 'Technology': technology, 'Capacity (KW)': capacity}, index = [i])
        df_capacity_general=df_capacity_general.append(new_row, ignore_index = False)

#### Saving to csv

In [7]:
df_capacity.to_csv('treated_data/capacity.csv', sep=';', index=True)
df_capacity_general.to_csv('treated_data/capacity_general.csv')

In [8]:
df_capacity = pd.read_csv('treated_data/Capacity/capacity.csv',sep=';').drop('Unnamed: 0', axis=1)

# Merge Production & Capacity on Year 
#### Il manque encore (1) comprendre quelle est l'unité de capacity (2) capacity factor (je ne peux pas trouver
#### quelque chose qui dit la durée max dans les excels.

In [13]:
for energy in energy_infos:
    
    df_production = pd.read_csv(energy_infos[energy][0],sep = ';').drop('Unnamed: 0', axis=1)
    df_production = df_production.where(df_production['Year']>2000).dropna()
    
    df_capacity = pd.read_csv('treated_data/Capacity/capacity_general.csv').drop('Unnamed: 0', axis=1)
    df_capacity = df_capacity.where(df_capacity['Technology']==energy_infos[energy][1]).dropna()
    
    df_energy = pd.merge (df_production, df_capacity, on= 'Year')
    df_energy['Production (GWh)'] = df_energy['Value']
    df_energy['Capacity (MW)'] = df_energy['Capacity']
    df_energy.drop(['Value','Capacity'], axis = 1, inplace = True)
    df_energy['Capacity Factor (%)'] = 100 * (df_energy['Production (GWh)']*1000000)/(df_energy['Capacity (MW)']*365*24)
    
    path = 'treated_data/Energies/' + energy + '_treated.csv'
    df_energy.to_csv(path, sep = ';')